In [1]:
import numpy as np
import cv2

In [4]:
in_w = 300
in_h = 300
whratio = in_w/float(in_h)
inScaleFactor = 0.007
meanVal  = 127

video = None
Caffe = False
if Caffe:
    prototext = r"C:\Users\saket\Desktop\courses\TF\OpenCV\datasets\SSD\MobileNetSSD_deploy.prototxt"
    weights = r"C:\Users\saket\Desktop\courses\TF\OpenCV\datasets\SSD\MobileNetSSD_deploy.caffemodel"
    n_classes = 20
else:
    prototext = r"C:\Users\saket\Desktop\courses\TF\OpenCV\datasets\SSD\ssd_mobilenet_v1_coco.pbtxt"
    weights = r"C:\Users\saket\Desktop\courses\TF\OpenCV\datasets\SSD\frozen_inference_graph.pb"
    n_classes = 90
thresh = 0.2


In [5]:
if __name__ =="__main__":

    if n_classes == 20:
        net = cv2.dnn.readNetFromCaffe(prototext,weights)
        swapRB = False
        className = { 0: 'background',
            1: 'aeroplane', 2: 'bicycle', 3: 'bird', 4: 'boat',
            5: 'bottle', 6: 'bus', 7: 'car', 8: 'cat', 9: 'chair',
            10: 'cow', 11: 'diningtable', 12: 'dog', 13: 'horse',
            14: 'motorbike', 15: 'person', 16: 'pottedplant',
            17: 'sheep', 18: 'sofa', 19: 'train', 20: 'tvmonitor' }
    else:
        assert(n_classes == 90)
        net = cv2.dnn.readNetFromTensorflow(weights,prototext)
        swapRB = True
        className = { 0: 'background',
            1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus',
            7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant',
            13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat',
            18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear',
            24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag',
            32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard',
            37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove',
            41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle',
            46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon',
            51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange',
            56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut',
            61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed',
            67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse',
            75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'microwave', 79: 'oven',
            80: 'toaster', 81: 'sink', 82: 'refrigerator', 84: 'book', 85: 'clock',
            86: 'vase', 87: 'scissors', 88: 'teddy bear', 89: 'hair drier', 90: 'toothbrush' }
 

In [7]:
if video:
    cam = cv2.VideoCapture(video)
else:
    came = cv2.VideoCapture(1)

while True:
    ret,frame = came.read()
    blob = cv2.dnn.blobFromImage(frame,inScaleFactor,(in_w,in_h),(meanVal, meanVal, meanVal), swapRB)
    net.setInput(blob)
    detect = net.forward()

    cols = frame.shape[1]
    rows = frame.shape[0]

    if cols/float(rows) > whratio:
        cropsize = (int(rows*whratio),rows)
    else:
        cropsize = (cols,int(cols/whratio))
    
    y1 = int((rows - cropsize[1])/2)
    y2 = y1+cropsize[1]
    x1 = int((cols - cropsize[0])/2)
    x2 = x1+cropsize[0]
    frame = frame[y1:y2,x1:x2]

    cols = frame.shape[1]
    rows = frame.shape[0]

    for i in range(detect.shape[2]):
        conf = detect[0,0,i,2]
        if conf>thresh:
            class_id = int(detect[0,0,i,1])

            xleftbottom = int(detect[0,0,i,3]*cols)
            yleftbottom = int(detect[0,0,i,4]*cols)
            xrighttop = int(detect[0,0,i,5]*cols)
            yrighttop = int(detect[0,0,i,6]*rows)

            cv2.rectangle(frame,(xleftbottom,yleftbottom),(xrighttop,yrighttop),(0,255,0))
            if class_id in className:
                label = className[class_id]+": " +str(conf)
                labelSize,base = cv2.getTextSize(label,cv2.FONT_HERSHEY_COMPLEX,0.5,1)

                yleftbottom = max(yleftbottom,labelSize[1])
                cv2.rectangle(frame,(xleftbottom,yleftbottom - labelSize[1]),(xleftbottom+labelSize[0],yleftbottom+base)
                ,(255,255,255),cv2.FILLED)

                cv2.putText(frame,label,(xleftbottom,yleftbottom),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0))
    
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1)>=0:
        break
came.release()
cv2.destroyAllWindows()

